# University of Aberdeen

## Applied AI (CS5079)

### Resit CA1 - Reinforcement Learning with OpenAI Gym

In [2]:
#Library for environments
import gym
from gym.envs.registration import register
from gym.envs.toy_text.frozen_lake import generate_random_map

#Librairies to represent the output
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

#Essential libraries for computation
import numpy as np
import random
import tensorflow as tf
from collections import deque

We will use the following environment.

In [ ]:
random_map = generate_random_map(size=10, p=0.3)
env = gym.make("FrozenLake-v1", desc=random_map, is_slippery=True)
env.reset()
env.render()

In [13]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Task 1.2: Define a Neural Network-based Agent (DQN)
class DQN(nn.Module):
    def __init__(self, input_shape, out_actions):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_shape, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_actions)
        )

    def forward(self, x):
        return self.fc(x)

# Replay Memory for Experience Replay
class ReplayMemory:
    def __init__(self, maxlen):
        self.memory = deque(maxlen=maxlen)

    def append(self, transition):
        self.memory.append(transition)

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

# Task 1.2 & 1.3: DQN Agent Implementation
class FrozenLakeDQL:
    def __init__(self, env, episodes=5000, gamma=0.99, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995, learning_rate=0.001, batch_size=64, memory_size=10000):
        self.env = env
        self.episodes = episodes
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.memory = ReplayMemory(memory_size)

        # Initialize policy and target networks
        # Use one-hot encoding for states
        self.state_size = env.observation_space.n
        self.action_size = env.action_space.n
        self.policy_net = DQN(self.state_size, self.action_size)
        self.target_net = DQN(self.state_size, self.action_size)
        self.target_net.load_state_dict(self.policy_net.state_dict())

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)

    def _one_hot_encode(self, state):
        # Convert state to one-hot encoded tensor
        one_hot = torch.zeros(self.state_size)
        one_hot[state] = 1.0
        return one_hot

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            # Convert state to one-hot encoded tensor
            state_tensor = self._one_hot_encode(state).unsqueeze(0)
            with torch.no_grad():
                return self.policy_net(state_tensor).argmax().item()

    def replay_experience(self):
        if len(self.memory.memory) < self.batch_size:
            return

        batch = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # One-hot encode states and next states
        states_tensor = torch.stack([self._one_hot_encode(s) for s in states])
        next_states_tensor = torch.stack([self._one_hot_encode(s) for s in next_states])
        
        actions_tensor = torch.LongTensor(actions).unsqueeze(1)
        rewards_tensor = torch.FloatTensor(rewards)
        dones_tensor = torch.FloatTensor(dones)

        # Q-Learning target
        with torch.no_grad():
            target_q_values = rewards_tensor + self.gamma * (1 - dones_tensor) * self.target_net(next_states_tensor).max(1)[0]

        # Current Q-values
        current_q_values = self.policy_net(states_tensor).gather(1, actions_tensor).squeeze()

        # Compute loss
        loss = nn.MSELoss()(current_q_values, target_q_values)

        # Optimize the policy network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def train(self):
        rewards = []
        for episode in range(self.episodes):
            state, _ = self.env.reset()
            done = False
            total_reward = 0

            while not done:
                action = self.choose_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                self.memory.append((state, action, reward, next_state, done))
                self.replay_experience()

                state = next_state
                total_reward += reward

            rewards.append(total_reward)

            # Decay epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            # Sync target network with policy network every 10 episodes
            if episode % 10 == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict())

            print(f"Episode {episode + 1}/{self.episodes}, Total Reward: {total_reward}, Epsilon: {self.epsilon:.2f}")

        return rewards

    def test(self, episodes=100):
        success_count = 0
        for _ in range(episodes):
            state, _ = self.env.reset()
            done = False
            while not done:
                # Convert state to one-hot encoded tensor
                state_tensor = self._one_hot_encode(state).unsqueeze(0)
                with torch.no_grad():
                    action = self.policy_net(state_tensor).argmax().item()
                state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                if reward == 1:
                    success_count += 1
        print(f"Test Success Rate: {success_count / episodes * 100:.2f}%")
        return success_count / episodes

# Task 1.4: Informed Agent Implementation remains the same
class InformedAgent:
    def __init__(self, map_size):
        self.map_size = map_size
        self.goal_position = (map_size - 1, map_size - 1)

    def get_action(self, state):
        row, col = state // self.map_size, state % self.map_size
        if row < self.goal_position[0]:
            return 1
        elif col < self.goal_position[1]:
            return 2
        elif row > self.goal_position[0]:
            return 3
        elif col > self.goal_position[1]:
            return 0

    def test(self, env, episodes=5000):
        success_count = 0
        for _ in range(episodes):
            state, _ = env.reset()
            done = False
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                if reward == 1:
                    success_count += 1
        print(f"Informed Agent Success Rate: {success_count / episodes * 100:.2f}%")
        return success_count / episodes

# Main Function
def main():
    env = gym.make('FrozenLake-v1', map_name="8x8", is_slippery=True)

    # Task 1.2 and 1.3: Train DQN Agent
    dqn_agent = FrozenLakeDQL(env)
    rewards = dqn_agent.train()

    # Task 1.3: Test DQN Agent
    print("\nTesting DQN Agent:")
    dqn_agent.test()

    # Task 1.4: Test Informed Agent
    print("\nTesting Informed Agent:")
    informed_agent = InformedAgent(map_size=8)
    informed_agent.test(env)

if __name__ == '__main__':
    main()

Episode 1/5000, Total Reward: 0.0, Epsilon: 0.99
Episode 2/5000, Total Reward: 0.0, Epsilon: 0.99
Episode 3/5000, Total Reward: 0.0, Epsilon: 0.99
Episode 4/5000, Total Reward: 0.0, Epsilon: 0.98
Episode 5/5000, Total Reward: 0.0, Epsilon: 0.98
Episode 6/5000, Total Reward: 0.0, Epsilon: 0.97
Episode 7/5000, Total Reward: 0.0, Epsilon: 0.97
Episode 8/5000, Total Reward: 0.0, Epsilon: 0.96
Episode 9/5000, Total Reward: 0.0, Epsilon: 0.96
Episode 10/5000, Total Reward: 0.0, Epsilon: 0.95
Episode 11/5000, Total Reward: 0.0, Epsilon: 0.95
Episode 12/5000, Total Reward: 0.0, Epsilon: 0.94
Episode 13/5000, Total Reward: 0.0, Epsilon: 0.94
Episode 14/5000, Total Reward: 0.0, Epsilon: 0.93
Episode 15/5000, Total Reward: 0.0, Epsilon: 0.93
Episode 16/5000, Total Reward: 0.0, Epsilon: 0.92
Episode 17/5000, Total Reward: 0.0, Epsilon: 0.92
Episode 18/5000, Total Reward: 0.0, Epsilon: 0.91
Episode 19/5000, Total Reward: 0.0, Epsilon: 0.91
Episode 20/5000, Total Reward: 0.0, Epsilon: 0.90
Episode 2

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import random
import io
import base64
from IPython.display import HTML
import os
from pathlib import Path

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
# Function to generate a random FrozenLake map
from gym.envs.toy_text.frozen_lake import generate_random_map

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Task 1.2: Define a Neural Network-based Agent (DQN)
class DQN(nn.Module):
    def __init__(self, input_shape, out_actions):
        super(DQN, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_shape, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_actions)
        )

    def forward(self, x):
        return self.fc(x)

# Replay Memory for Experience Replay
class ReplayMemory:
    def __init__(self, maxlen):
        self.memory = deque(maxlen=maxlen)

    def append(self, transition):
        self.memory.append(transition)

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

# Task 1.2 & 1.3: DQN Agent Implementation
class FrozenLakeDQL:
    def __init__(self, env, episodes=5000, gamma=0.99, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995, learning_rate=0.001, batch_size=64, memory_size=10000):
        self.env = env
        self.episodes = episodes
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.memory = ReplayMemory(memory_size)

        # Initialize policy and target networks
        # Use one-hot encoding for states
        self.state_size = env.observation_space.n
        self.action_size = env.action_space.n
        self.policy_net = DQN(self.state_size, self.action_size)
        self.target_net = DQN(self.state_size, self.action_size)
        self.target_net.load_state_dict(self.policy_net.state_dict())

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.learning_rate)

    def _one_hot_encode(self, state):
        # Convert state to one-hot encoded tensor
        one_hot = torch.zeros(self.state_size)
        one_hot[state] = 1.0
        return one_hot

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return self.env.action_space.sample()
        else:
            # Convert state to one-hot encoded tensor
            state_tensor = self._one_hot_encode(state).unsqueeze(0)
            with torch.no_grad():
                return self.policy_net(state_tensor).argmax().item()

    def replay_experience(self):
        if len(self.memory.memory) < self.batch_size:
            return

        batch = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)

        # One-hot encode states and next states
        states_tensor = torch.stack([self._one_hot_encode(s) for s in states])
        next_states_tensor = torch.stack([self._one_hot_encode(s) for s in next_states])
        
        actions_tensor = torch.LongTensor(actions).unsqueeze(1)
        rewards_tensor = torch.FloatTensor(rewards)
        dones_tensor = torch.FloatTensor(dones)

        # Q-Learning target
        with torch.no_grad():
            target_q_values = rewards_tensor + self.gamma * (1 - dones_tensor) * self.target_net(next_states_tensor).max(1)[0]

        # Current Q-values
        current_q_values = self.policy_net(states_tensor).gather(1, actions_tensor).squeeze()

        # Compute loss
        loss = nn.MSELoss()(current_q_values, target_q_values)

        # Optimize the policy network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def train(self):
        rewards = []
        for episode in range(self.episodes):
            state, _ = self.env.reset()
            done = False
            total_reward = 0

            while not done:
                action = self.choose_action(state)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                self.memory.append((state, action, reward, next_state, done))
                self.replay_experience()

                state = next_state
                total_reward += reward

            rewards.append(total_reward)

            # Decay epsilon
            self.epsilon = max(self.epsilon_min, self.epsilon * self.epsilon_decay)

            # Sync target network with policy network every 10 episodes
            if episode % 10 == 0:
                self.target_net.load_state_dict(self.policy_net.state_dict())

            print(f"Episode {episode + 1}/{self.episodes}, Total Reward: {total_reward}, Epsilon: {self.epsilon:.2f}")

        return rewards

    def test(self, episodes=100):
        success_count = 0
        for _ in range(episodes):
            state, _ = self.env.reset()
            done = False
            while not done:
                # Convert state to one-hot encoded tensor
                state_tensor = self._one_hot_encode(state).unsqueeze(0)
                with torch.no_grad():
                    action = self.policy_net(state_tensor).argmax().item()
                state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                if reward == 1:
                    success_count += 1
        print(f"Test Success Rate: {success_count / episodes * 100:.2f}%")
        return success_count / episodes

    def record_episode(self, video_folder='./video2'):
        """
        Record a video of the trained agent playing an episode
        
        Parameters:
        - video_folder: Folder to save the video
        
        Returns:
        - Total reward obtained during the episode
        """
        # Create a new environment with video recording
        env = gym.make("FrozenLake-v1", map_name="10x10", is_slippery=True, render_mode="rgb_array")
        env = gym.wrappers.RecordVideo(env, video_folder=video_folder, 
                                       episode_trigger=lambda x: True)
        
        # Reset the environment
        state, _ = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            # Choose action with current policy
            action = self.choose_action(state)
            
            # Take action in the environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            total_reward += reward
            state = next_state
        
        # Close the environment
        env.close()
        
        print(f"Total Reward in recorded episode: {total_reward}")
        
        return total_reward

    @staticmethod
    def display_video(video_path=None):
        """
        Display the most recently recorded video
        
        Parameters:
        - video_path: Optional specific video path. 
                      If None, finds the most recent MP4 in ./video2
        
        Returns:
        - HTML video player
        """
        # If no specific path provided, find the most recent video
        if video_path is None:
            video_folder = Path('./video2')
            video_path = sorted(video_folder.glob("*.mp4"), key=os.path.getmtime)[-1]
        
        # Read the video file
        with open(video_path, 'rb') as video_file:
            video = video_file.read()
        
        # Encode video for HTML display
        encoded = base64.b64encode(video).decode('ascii')
        
        # Return HTML video player
        return HTML(data=f'''
            <video width="640" height="480" controls>
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
            </video>
            ''')

# Task 1.4: Informed Agent Implementation remains the same
class InformedAgent:
    def __init__(self, map_size):
        self.map_size = map_size
        self.goal_position = (map_size - 1, map_size - 1)

    def get_action(self, state):
        row, col = state // self.map_size, state % self.map_size
        if row < self.goal_position[0]:
            return 1
        elif col < self.goal_position[1]:
            return 2
        elif row > self.goal_position[0]:
            return 3
        elif col > self.goal_position[1]:
            return 0

    def test(self, env, episodes=50000):
        success_count = 0
        for _ in range(episodes):
            state, _ = env.reset()
            done = False
            while not done:
                action = self.get_action(state)
                state, reward, terminated, truncated, _ = env.step(action)
                done = terminated or truncated
                if reward == 1:
                    success_count += 1
        print(f"Informed Agent Success Rate: {success_count / episodes * 100:.2f}%")
        return success_count / episodes

# Main Function
def main():
    # Generate a random map for the Frozen Lake environment
    random_map = generate_random_map(size=10, p=0.3)
    
    # Create the Frozen Lake environment with the generated random map
    env = gym.make("FrozenLake-v1", desc=random_map, is_slippery=True)

    # Render the environment to visualize the generated map
    env.reset()
    env.render()

    # Task 1.2 and 1.3: Train DQN Agent
    dqn_agent = FrozenLakeDQL(env)
    rewards = dqn_agent.train()

    # Task 1.3: Test DQN Agent
    print("\nTesting DQN Agent:")
    dqn_agent.test()

    # Optional: Record and display a video of the trained agent
    print("\nRecording Episode Video:")
    dqn_agent.record_episode()
    # Note: To display the video, you would call FrozenLakeDQL.display_video() 
    # in a Jupyter notebook after this script runs

    # Task 1.4: Test Informed Agent
    print("\nTesting Informed Agent:")
    informed_agent = InformedAgent(map_size=10)
    informed_agent.test(env)

if __name__ == '__main__':
    main()


Episode 1/5000, Total Reward: 0.0, Epsilon: 0.99
Episode 2/5000, Total Reward: 0.0, Epsilon: 0.99
Episode 3/5000, Total Reward: 0.0, Epsilon: 0.99
Episode 4/5000, Total Reward: 0.0, Epsilon: 0.98
Episode 5/5000, Total Reward: 0.0, Epsilon: 0.98
Episode 6/5000, Total Reward: 0.0, Epsilon: 0.97
Episode 7/5000, Total Reward: 0.0, Epsilon: 0.97
Episode 8/5000, Total Reward: 0.0, Epsilon: 0.96
Episode 9/5000, Total Reward: 0.0, Epsilon: 0.96
Episode 10/5000, Total Reward: 0.0, Epsilon: 0.95
Episode 11/5000, Total Reward: 0.0, Epsilon: 0.95
Episode 12/5000, Total Reward: 0.0, Epsilon: 0.94
Episode 13/5000, Total Reward: 0.0, Epsilon: 0.94
Episode 14/5000, Total Reward: 0.0, Epsilon: 0.93
Episode 15/5000, Total Reward: 0.0, Epsilon: 0.93
Episode 16/5000, Total Reward: 0.0, Epsilon: 0.92
Episode 17/5000, Total Reward: 0.0, Epsilon: 0.92
Episode 18/5000, Total Reward: 0.0, Epsilon: 0.91
Episode 19/5000, Total Reward: 0.0, Epsilon: 0.91
Episode 20/5000, Total Reward: 0.0, Epsilon: 0.90
Episode 2

TypeError: FrozenLakeEnv.__init__() got an unexpected keyword argument 'p'

In [21]:
FrozenLakeDQL.display_video()

In [1]:
%env KMP_DUPLICATE_LIB_OK=TRUE

env: KMP_DUPLICATE_LIB_OK=TRUE


In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
from gym.wrappers import RecordVideo
import os
import heapq
import time

# Create FrozenLake Environment with random 10x10 map
from gym.envs.toy_text.frozen_lake import generate_random_map

# Hyperparameters with detailed explanations
learning_rate = 0.001  # Controls the step size at each iteration while moving toward a minimum of the loss function
gamma = 0.99  # Discount factor: Determines the importance of future rewards
epsilon = 1.0  # Start with full exploration
epsilon_decay = 0.995  # Gradually reduce random exploration
epsilon_min = 0.01  # Minimum exploration rate to ensure some randomness

# Neural network training parameters
batch_size = 256  # Number of experiences to sample for each training iteration
memory_size = 10000  # Size of experience replay buffer
num_episodes = 5000  # Reduced from 70M for practical training time

# Q-Network: Approximates the Q-value function using a neural network
class QNetwork(nn.Module):
    def __init__(self, state_size, action_size):
        super(QNetwork, self).__init__()
        # Three-layer neural network with ReLU activation
        self.fc1 = nn.Linear(state_size, 256)  # First hidden layer
        self.fc2 = nn.Linear(256, 256)  # Second hidden layer
        self.fc3 = nn.Linear(256, action_size)  # Output layer mapping to action values

    def forward(self, x):
        # ReLU activation helps introduce non-linearity and prevent vanishing gradient
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# DQN Agent: Implements Deep Q-Learning algorithm
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        
        # Experience replay: Store and randomly sample past experiences
        self.memory = deque(maxlen=memory_size)
        
        # Neural network to approximate Q-values
        self.model = QNetwork(state_size, action_size)
        
        # Adam optimizer: Adaptive learning rate for each parameter
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)
        
        # Mean Squared Error loss: Measures difference between predicted and target Q-values
        self.criterion = nn.MSELoss()
        
        # Exploration rate
        self.epsilon = epsilon

    def remember(self, state, action, reward, next_state, done):
        """Store experience in replay memory"""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """Action selection with epsilon-greedy strategy"""
        # Explore: Random action
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        # Exploit: Best known action
        state = torch.FloatTensor(state)
        with torch.no_grad():
            action_values = self.model(state)
        return torch.argmax(action_values).item()

    def update_epsilon(self):
        """Decay exploration rate"""
        if self.epsilon > epsilon_min:
            self.epsilon *= epsilon_decay

# A* Search for Guided Exploration
def heuristic(pos, goal):
    """Manhattan distance heuristic for efficient path finding"""
    return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

def a_star_search(env_desc):
    """Find potential path through the environment"""
    start = (0, 0)
    goal = (len(env_desc) - 1, len(env_desc[0]) - 1)
    frontier = [(0, start)]
    heapq.heapify(frontier)
    came_from = {start: None}
    cost_so_far = {start: 0}
    actions = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, Down, Left, Right

    while frontier:
        _, current = heapq.heappop(frontier)

        if current == goal:
            break

        for i, (dr, dc) in enumerate(directions):
            next_pos = (current[0] + dr, current[1] + dc)
            # Avoid holes while exploring
            if (0 <= next_pos[0] < len(env_desc) and 
                0 <= next_pos[1] < len(env_desc[0]) and 
                env_desc[next_pos[0]][next_pos[1]] != 'H'):
                new_cost = cost_so_far[current] + 1
                if next_pos not in cost_so_far or new_cost < cost_so_far[next_pos]:
                    cost_so_far[next_pos] = new_cost
                    priority = new_cost + heuristic(next_pos, goal)
                    heapq.heappush(frontier, (priority, next_pos))
                    came_from[next_pos] = current
                    actions.append(i)

    return actions

# Main Training Loop
def train_agent():
    # Generate random environment
    random_map = generate_random_map(size=10, p=0.3)
    env = gym.make("FrozenLake-v1", desc=random_map, is_slippery=False)

    # Initialize agent
    state_size = env.observation_space.n
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    
    # A* guided actions
    a_star_actions = a_star_search(env.desc)
    
    # Tracking metrics
    rewards = []
    moving_avg_rewards = []
    epsilon_values = []
    
    # A* action index tracker
    a_star_index = 0
    
    # Training
    for e in range(num_episodes):
        # Reset environment
        state, _ = env.reset()
        state = np.identity(state_size)[state]  # One-hot encode
        done = False
        total_reward = 0
        
        while not done:
            # Hybrid exploration: Mix A* and DQN actions
            if a_star_index < len(a_star_actions) and np.random.rand() < 0.5:
                action = a_star_actions[a_star_index]
                a_star_index += 1
            else:
                action = agent.act(state)

            # Take action
            next_state, reward, done, _, _ = env.step(action)
            next_state = np.identity(state_size)[next_state]
            
            # Store experience
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward

            # Train if enough memories collected
            if len(agent.memory) >= batch_size:
                minibatch = random.sample(agent.memory, batch_size)
                states, actions, rewards_batch, next_states, dones = zip(*minibatch)

                # Convert to tensors
                states = torch.FloatTensor(np.vstack(states))
                actions = torch.LongTensor(actions).unsqueeze(1)
                rewards_batch = torch.FloatTensor(rewards_batch).unsqueeze(1)
                next_states = torch.FloatTensor(np.vstack(next_states))
                dones = torch.FloatTensor(dones).unsqueeze(1)

                # Compute target Q-values
                with torch.no_grad():
                    max_next_qs = agent.model(next_states).max(1, keepdim=True)[0]
                    target_qs = rewards_batch + (1 - dones) * gamma * max_next_qs

                # Compute current Q-values
                current_qs = agent.model(states).gather(1, actions)

                # Compute loss and train
                loss = agent.criterion(current_qs, target_qs)
                agent.optimizer.zero_grad()
                loss.backward()
                agent.optimizer.step()

        # Update exploration
        agent.update_epsilon()
        rewards.append(total_reward)
        epsilon_values.append(agent.epsilon)

        # Calculate moving average reward
        if len(rewards) >= 100:
            moving_avg_rewards.append(np.mean(rewards[-100:]))
        else:
            moving_avg_rewards.append(np.mean(rewards))

        # Progress tracking
        if (e + 1) % 100 == 0:
            print(f"Episode {e+1}/{num_episodes}, Reward: {total_reward}, Epsilon: {agent.epsilon:.2f}")

    # Plotting results
    plt.figure(figsize=(15,5))
    
    # Reward Progression
    plt.subplot(131)
    plt.plot(moving_avg_rewards)
    plt.title('Moving Average Rewards')
    plt.xlabel('Episode')
    plt.ylabel('Average Reward')
    
    # Epsilon Decay
    plt.subplot(132)
    plt.plot(epsilon_values)
    plt.title('Epsilon Exploration Rate')
    plt.xlabel('Episode')
    plt.ylabel('Epsilon')
    
    # Reward Distribution
    plt.subplot(133)
    plt.hist(rewards, bins=20)
    plt.title('Reward Distribution')
    plt.xlabel('Reward')
    plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.savefig('training_metrics.png')
    plt.close()

    # Save the trained model
    torch.save(agent.model.state_dict(), 'frozen_lake_dqn_model.pth')

    return agent, rewards, moving_avg_rewards, random_map

# Video Recording Function
def record_successful_episodes(agent, random_map, num_episodes=100, max_record_episodes=10):
    """
    Record videos of successful episodes
    
    Parameters:
    - agent: Trained DQN agent
    - random_map: The map used for consistent environment
    - num_episodes: Number of episodes to test
    - max_record_episodes: Maximum number of successful episodes to record
    """
    # Create directory for video recordings
    video_dir = 'successful_episodes'
    os.makedirs(video_dir, exist_ok=True)
    
    # Tracking successful recordings
    successful_recordings = 0
    
    for episode in range(num_episodes):
        # Stop if we've recorded enough successful episodes
        if successful_recordings >= max_record_episodes:
            break
        
        # Create environment with video recording
        env = gym.make("FrozenLake-v1", 
                       render_mode="rgb_array", 
                       desc=random_map,
                       is_slippery=True)
        env = RecordVideo(env, video_dir, 
                          name_prefix=f'successful_episode_{episode}')
        
        # Reset environment
        state, _ = env.reset()
        state = np.identity(env.observation_space.n)[state]
        done = False
        total_reward = 0
        
        while not done:
            # Choose action using trained agent
            action = agent.act(state)
            
            # Take action
            next_state, reward, done, truncated, _ = env.step(action)
            
            # Update total reward
            total_reward += reward
            
            # One-hot encode next state
            next_state = np.identity(env.observation_space.n)[next_state]
            state = next_state
            
            # Check if episode was successful
            if done and reward > 0:
                print(f"Successful Episode {episode}: Reward = {total_reward}")
                successful_recordings += 1
                break
            
            # End episode if it takes too many steps
            if done:
                break
        
        # Close the environment
        env.close()
    
    print(f"Recorded {successful_recordings} successful episodes")
    
    # Create a video preview montage
    video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
    
    # Display video filenames
    print("\nRecorded Video Files:")
    for video in video_files:
        print(video)

# Jupyter Notebook Video Display Function
def display_videos_in_notebook(directory='successful_episodes'):
    """
    Display recorded videos in a Jupyter Notebook
    
    Parameters:
    - directory: Directory containing video files
    """
    from IPython.display import display, HTML
    import os
    
    video_files = [f for f in os.listdir(directory) if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(directory, video_file)
        video_html = f"""
        <video width="640" height="480" controls>
            <source src="{video_path}" type="video/mp4">
            Your browser does not support the video tag.
        </video>
        """
        display(HTML(video_html))

# Main execution
def main():
    # Train the agent
    agent, rewards, moving_avg_rewards, random_map = train_agent()
    
    # Record successful episodes
    record_successful_episodes(agent, random_map)

if __name__ == "__main__":
    main()

# Notebook usage instructions:
"""
# In Jupyter Notebook:
# 1. Run the training script
# 2. Run the video recording script
# 3. Use the following to display videos:
# display_videos_in_notebook()
"""

C:\Users\paako\AppData\Roaming\Python\Python312\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode 100/5000, Reward: 0.0, Epsilon: 0.61
Episode 200/5000, Reward: 0.0, Epsilon: 0.37
Episode 300/5000, Reward: 0.0, Epsilon: 0.22
Episode 400/5000, Reward: 0.0, Epsilon: 0.13
Episode 500/5000, Reward: 0.0, Epsilon: 0.08
Episode 600/5000, Reward: 0.0, Epsilon: 0.05
Episode 700/5000, Reward: 0.0, Epsilon: 0.03
Episode 800/5000, Reward: 0.0, Epsilon: 0.02
Episode 900/5000, Reward: 0.0, Epsilon: 0.01
Episode 1000/5000, Reward: 0.0, Epsilon: 0.01
Episode 1100/5000, Reward: 0.0, Epsilon: 0.01
Episode 1200/5000, Reward: 0.0, Epsilon: 0.01
Episode 1300/5000, Reward: 0.0, Epsilon: 0.01
Episode 1400/5000, Reward: 0.0, Epsilon: 0.01
Episode 1500/5000, Reward: 0.0, Epsilon: 0.01
Episode 1600/5000, Reward: 0.0, Epsilon: 0.01
Episode 1700/5000, Reward: 0.0, Epsilon: 0.01
Episode 1800/5000, Reward: 0.0, Epsilon: 0.01
Episode 1900/5000, Reward: 0.0, Epsilon: 0.01
Episode 2000/5000, Reward: 0.0, Epsilon: 0.01
Episode 2100/5000, Reward: 0.0, Epsilon: 0.01
Episode 2200/5000, Reward: 0.0, Epsilon: 0.

C:\Users\paako\AppData\Roaming\Python\Python312\site-packages\gym\wrappers\record_video.py:75: UserWarning: WARN: Overwriting existing videos at c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


MoviePy - Building video c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_0-episode-0.mp4.
MoviePy - Writing video c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_0-episode-0.mp4



MoviePy - Done !
MoviePy - video ready c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_0-episode-0.mp4
MoviePy - Building video c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_1-episode-0.mp4.
MoviePy - Writing video c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_1-episode-0.mp4



...

MoviePy - Done !
MoviePy - video ready c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_98-episode-0.mp4


MoviePy - Building video c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_99-episode-0.mp4.
MoviePy - Writing video c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_99-episode-0.mp4



MoviePy - Done !
MoviePy - video ready c:\Users\paako\Documents\CS5079 (2024-25) Applied Artificial Intelligence\Groupwork Assignment\successful_episodes\successful_episode_99-episode-0.mp4


Recorded 0 successful episodes

Recorded Video Files:
successful_episode_0-episode-0.mp4
successful_episode_1-episode-0.mp4
successful_episode_10-episode-0.mp4
successful_episode_11-episode-0.mp4
successful_episode_12-episode-0.mp4
successful_episode_13-episode-0.mp4
successful_episode_14-episode-0.mp4
successful_episode_15-episode-0.mp4
successful_episode_16-episode-0.mp4
successful_episode_17-episode-0.mp4
successful_episode_18-episode-0.mp4
successful_episode_19-episode-0.mp4
successful_episode_2-episode-0.mp4
successful_episode_20-episode-0.mp4
successful_episode_21-episode-0.mp4
successful_episode_22-episode-0.mp4
successful_episode_23-episode-0.mp4
successful_episode_24-episode-0.mp4
successful_episode_25-episode-0.mp4
successful_episode_26-episode-0.mp4
successful_episode_27-episode-0.mp4
successful_episode_28-episode-0.mp4
successful_episode_29-episode-0.mp4
successful_episode_3-episode-0.mp4
successful_episode_30-episode-0.mp4
successful_episode_31-episode-0.mp4
successful_epi

'\n# In Jupyter Notebook:\n# 1. Run the training script\n# 2. Run the video recording script\n# 3. Use the following to display videos:\n# display_videos_in_notebook()\n'

In [5]:
display_videos_in_notebook()